<a href="https://colab.research.google.com/github/vonvolous/d2l/blob/main/D2l_7_LeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

parameter 설정

In [3]:
lr = 1e-3
batch_size = 64
epochs = 20

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

dataset 준비

In [5]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor()
])
trainset = torchvision.datasets.MNIST(root='./datasets',
                                      transform=transform,
                                      train=True,
                                      download=True)
testset = torchvision.datasets.MNIST(root='./datasets',
                                     transform=transform,
                                     train=False,
                                     download=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 284934935.50it/s]

Extracting ./datasets/MNIST/raw/train-images-idx3-ubyte.gz to ./datasets/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 82912863.67it/s]


Extracting ./datasets/MNIST/raw/train-labels-idx1-ubyte.gz to ./datasets/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 178391750.84it/s]

Extracting ./datasets/MNIST/raw/t10k-images-idx3-ubyte.gz to ./datasets/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 16158209.30it/s]


Extracting ./datasets/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./datasets/MNIST/raw



In [6]:
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False)

model

In [11]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.C1 = nn.Conv2d(1, 6, kernel_size=5)
        self.S2 = nn.AvgPool2d(2)
        self.C3 = nn.Conv2d(6, 16, kernel_size=5)
        self.S4 = nn.AvgPool2d(2)
        self.C5 = nn.Conv2d(16, 120, kernel_size=5)
        self.F6 = nn.Linear(120,84)
        self.Out = nn.Linear(84,10)
    def forward(self, x):
        x = F.sigmoid(self.C1(x))
        x = self.S2(x)
        x = F.sigmoid(self.C3(x))
        x = self.S4(x)
        x = F.sigmoid(self.C5(x))
        x = F.sigmoid(self.F6(torch.squeeze(x)))
        x = self.Out(x)
        return x
model = LeNet().to(device)
model

LeNet(
  (C1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (S2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (C3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (S4): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (C5): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  (F6): Linear(in_features=120, out_features=84, bias=True)
  (Out): Linear(in_features=84, out_features=10, bias=True)
)

train

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [13]:
for epoch in range(epochs):
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch}/{epochs}] Loss: {loss.item():.4f}')

Epoch [0/20] Loss: 0.4937
Epoch [1/20] Loss: 0.1177
Epoch [2/20] Loss: 0.1702
Epoch [3/20] Loss: 0.1595
Epoch [4/20] Loss: 0.1822
Epoch [5/20] Loss: 0.0416
Epoch [6/20] Loss: 0.0143
Epoch [7/20] Loss: 0.0086
Epoch [8/20] Loss: 0.0577
Epoch [9/20] Loss: 0.0591
Epoch [10/20] Loss: 0.0139
Epoch [11/20] Loss: 0.0124
Epoch [12/20] Loss: 0.0015
Epoch [13/20] Loss: 0.0022
Epoch [14/20] Loss: 0.0144
Epoch [15/20] Loss: 0.0185
Epoch [16/20] Loss: 0.0298
Epoch [17/20] Loss: 0.0136
Epoch [18/20] Loss: 0.0019
Epoch [19/20] Loss: 0.0004


test